# Production RAG 2025: Deployment, Monitoring & Enterprise Features

Learn how to deploy, monitor, and scale RAG systems in production environments with enterprise-grade features.

## 🎯 What You'll Learn

- **Production Architecture** - Scalable system design patterns
- **Performance Monitoring** - Metrics, logging, and observability
- **Cost Optimization** - Resource management and efficiency
- **Security & Compliance** - Data protection and access control
- **Deployment Strategies** - CI/CD, containerization, cloud deployment
- **Enterprise Integration** - SSO, audit trails, enterprise connectors

## 📋 Prerequisites

- Completed previous RAG notebooks
- Understanding of cloud platforms (AWS/GCP/Azure)
- Basic DevOps knowledge

## 🏗️ Production Architecture Patterns

Let's explore different architectural approaches for production RAG systems.

In [ ]:
import os
import time
import logging
import json
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional, Union
from dataclasses import dataclass, field
from enum import Enum
import threading
from collections import defaultdict, deque
import uuid
import hashlib
from pathlib import Path

# Production monitoring and metrics
from dotenv import load_dotenv

# Load environment
load_dotenv()

print("🏭 Production RAG Setup")
print("=" * 40)

### 📊 Performance Monitoring System

In [ ]:
class MetricType(Enum):
    COUNTER = "counter"
    GAUGE = "gauge"
    HISTOGRAM = "histogram"
    TIMER = "timer"

@dataclass
class Metric:
    name: str
    value: float
    metric_type: MetricType
    tags: Dict[str, str] = field(default_factory=dict)
    timestamp: float = field(default_factory=time.time)

class RAGMetrics:
    """Comprehensive metrics collection for RAG systems."""
    
    def __init__(self):
        self.metrics = defaultdict(list)
        self.counters = defaultdict(int)
        self.gauges = defaultdict(float)
        self.histograms = defaultdict(list)
        self.timers = defaultdict(list)
        
        # Performance tracking
        self.query_times = deque(maxlen=1000)  # Last 1000 queries
        self.error_count = 0
        self.success_count = 0
        
        # Resource tracking
        self.api_calls = {
            "llm": 0,
            "embedding": 0,
            "vector_search": 0
        }
        
        self.costs = {
            "llm_tokens": 0,
            "embedding_tokens": 0,
            "estimated_cost": 0.0
        }
    
    def increment(self, metric_name: str, value: int = 1, tags: Dict[str, str] = None):
        """Increment a counter metric."""
        self.counters[metric_name] += value
        metric = Metric(metric_name, value, MetricType.COUNTER, tags or {})
        self.metrics[metric_name].append(metric)
    
    def gauge(self, metric_name: str, value: float, tags: Dict[str, str] = None):
        """Set a gauge metric."""
        self.gauges[metric_name] = value
        metric = Metric(metric_name, value, MetricType.GAUGE, tags or {})
        self.metrics[metric_name].append(metric)
    
    def histogram(self, metric_name: str, value: float, tags: Dict[str, str] = None):
        """Record a histogram value."""
        self.histograms[metric_name].append(value)
        metric = Metric(metric_name, value, MetricType.HISTOGRAM, tags or {})
        self.metrics[metric_name].append(metric)
    
    def timer(self, metric_name: str):
        """Context manager for timing operations."""
        return TimerContext(self, metric_name)
    
    def record_query_performance(self, duration: float, success: bool, tokens_used: int = 0):
        """Record query performance metrics."""
        self.query_times.append(duration)
        
        if success:
            self.success_count += 1
            self.increment("queries.success")
        else:
            self.error_count += 1
            self.increment("queries.error")
        
        self.histogram("query.duration", duration)
        
        if tokens_used > 0:
            self.costs["llm_tokens"] += tokens_used
            self.histogram("tokens.used", tokens_used)
    
    def get_summary(self) -> Dict[str, Any]:
        """Get metrics summary."""
        total_queries = self.success_count + self.error_count
        avg_duration = sum(self.query_times) / len(self.query_times) if self.query_times else 0
        
        return {
            "performance": {
                "total_queries": total_queries,
                "success_rate": self.success_count / total_queries if total_queries > 0 else 0,
                "avg_query_time": avg_duration,
                "p95_query_time": self._percentile(list(self.query_times), 95) if self.query_times else 0,
                "p99_query_time": self._percentile(list(self.query_times), 99) if self.query_times else 0
            },
            "api_usage": self.api_calls,
            "costs": self.costs,
            "counters": dict(self.counters),
            "gauges": dict(self.gauges)
        }
    
    def _percentile(self, data: List[float], percentile: float) -> float:
        """Calculate percentile from sorted data."""
        if not data:
            return 0
        
        sorted_data = sorted(data)
        index = int(len(sorted_data) * percentile / 100)
        return sorted_data[min(index, len(sorted_data) - 1)]

class TimerContext:
    """Context manager for timing operations."""
    
    def __init__(self, metrics: RAGMetrics, metric_name: str):
        self.metrics = metrics
        self.metric_name = metric_name
        self.start_time = None
    
    def __enter__(self):
        self.start_time = time.time()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        duration = time.time() - self.start_time
        self.metrics.histogram(self.metric_name, duration)
        self.metrics.timers[self.metric_name].append(duration)

# Initialize metrics system
rag_metrics = RAGMetrics()
print("📊 Metrics system initialized")

### 🔐 Security and Access Control

In [ ]:
from enum import Enum
from datetime import datetime, timedelta

class UserRole(Enum):
    ADMIN = "admin"
    USER = "user"
    READ_ONLY = "read_only"
    SERVICE = "service"

@dataclass
class User:
    id: str
    email: str
    role: UserRole
    permissions: List[str] = field(default_factory=list)
    last_login: Optional[datetime] = None
    created_at: datetime = field(default_factory=datetime.now)

@dataclass
class AuditLog:
    user_id: str
    action: str
    resource: str
    timestamp: datetime = field(default_factory=datetime.now)
    details: Dict[str, Any] = field(default_factory=dict)
    ip_address: Optional[str] = None
    user_agent: Optional[str] = None

class SecurityManager:
    """Security and access control for RAG systems."""
    
    def __init__(self):
        self.users = {}
        self.sessions = {}
        self.audit_logs = deque(maxlen=10000)  # Keep last 10k logs
        self.api_keys = {}
        self.rate_limits = defaultdict(deque)  # User ID -> request timestamps
        
        # Security policies
        self.session_timeout = timedelta(hours=24)
        self.max_requests_per_minute = 100
        self.password_policy = {
            "min_length": 8,
            "require_uppercase": True,
            "require_lowercase": True,
            "require_numbers": True,
            "require_special": True
        }
    
    def create_user(self, email: str, role: UserRole, permissions: List[str] = None) -> str:
        """Create a new user."""
        user_id = str(uuid.uuid4())
        user = User(
            id=user_id,
            email=email,
            role=role,
            permissions=permissions or self._default_permissions(role)
        )
        
        self.users[user_id] = user
        
        self._log_audit(
            user_id="system",
            action="create_user",
            resource=f"user:{user_id}",
            details={"email": email, "role": role.value}
        )
        
        return user_id
    
    def _default_permissions(self, role: UserRole) -> List[str]:
        """Get default permissions for a role."""
        permissions_map = {
            UserRole.ADMIN: ["query", "manage_users", "view_metrics", "manage_content"],
            UserRole.USER: ["query"],
            UserRole.READ_ONLY: ["view_metrics"],
            UserRole.SERVICE: ["query", "batch_operations"]
        }
        return permissions_map.get(role, [])
    
    def create_session(self, user_id: str) -> str:
        """Create a user session."""
        if user_id not in self.users:
            raise ValueError("User not found")
        
        session_token = str(uuid.uuid4())
        self.sessions[session_token] = {
            "user_id": user_id,
            "created_at": datetime.now(),
            "last_activity": datetime.now()
        }
        
        self.users[user_id].last_login = datetime.now()
        
        self._log_audit(
            user_id=user_id,
            action="login",
            resource="session"
        )
        
        return session_token
    
    def validate_session(self, session_token: str) -> Optional[User]:
        """Validate a session token."""
        if session_token not in self.sessions:
            return None
        
        session = self.sessions[session_token]
        
        # Check if session expired
        if datetime.now() - session["created_at"] > self.session_timeout:
            del self.sessions[session_token]
            return None
        
        # Update last activity
        session["last_activity"] = datetime.now()
        
        return self.users.get(session["user_id"])
    
    def check_rate_limit(self, user_id: str) -> bool:
        """Check if user is within rate limits."""
        now = time.time()
        minute_ago = now - 60
        
        # Clean old requests
        user_requests = self.rate_limits[user_id]
        while user_requests and user_requests[0] < minute_ago:
            user_requests.popleft()
        
        # Check limit
        if len(user_requests) >= self.max_requests_per_minute:
            return False
        
        # Add current request
        user_requests.append(now)
        return True
    
    def has_permission(self, user: User, permission: str) -> bool:
        """Check if user has specific permission."""
        return permission in user.permissions or user.role == UserRole.ADMIN
    
    def _log_audit(self, user_id: str, action: str, resource: str, details: Dict[str, Any] = None, ip_address: str = None):
        """Log audit event."""
        audit_log = AuditLog(
            user_id=user_id,
            action=action,
            resource=resource,
            details=details or {},
            ip_address=ip_address
        )
        self.audit_logs.append(audit_log)
    
    def get_audit_logs(self, user_id: str = None, hours: int = 24) -> List[AuditLog]:
        """Get audit logs for a time period."""
        cutoff = datetime.now() - timedelta(hours=hours)
        logs = [log for log in self.audit_logs if log.timestamp >= cutoff]
        
        if user_id:
            logs = [log for log in logs if log.user_id == user_id]
        
        return logs
    
    def generate_api_key(self, user_id: str, name: str) -> str:
        """Generate API key for a user."""
        if user_id not in self.users:
            raise ValueError("User not found")
        
        api_key = f"rag_{hashlib.md5(f'{user_id}_{name}_{time.time()}'.encode()).hexdigest()}"
        
        self.api_keys[api_key] = {
            "user_id": user_id,
            "name": name,
            "created_at": datetime.now(),
            "last_used": None
        }
        
        self._log_audit(
            user_id=user_id,
            action="create_api_key",
            resource="api_key",
            details={"name": name}
        )
        
        return api_key
    
    def validate_api_key(self, api_key: str) -> Optional[User]:
        """Validate API key and return associated user."""
        if api_key not in self.api_keys:
            return None
        
        key_info = self.api_keys[api_key]
        key_info["last_used"] = datetime.now()
        
        return self.users.get(key_info["user_id"])

# Initialize security manager
security_manager = SecurityManager()

# Create sample users
admin_id = security_manager.create_user("admin@company.com", UserRole.ADMIN)
user_id = security_manager.create_user("user@company.com", UserRole.USER)

print("🔐 Security system initialized")
print(f"   Created admin user: {admin_id}")
print(f"   Created regular user: {user_id}")

### 💰 Cost Management and Optimization

In [ ]:
@dataclass
class CostEvent:
    timestamp: datetime
    service: str
    operation: str
    cost: float
    tokens: int = 0
    user_id: str = ""
    details: Dict[str, Any] = field(default_factory=dict)

class CostManager:
    """Cost tracking and optimization for RAG systems."""
    
    def __init__(self):
        self.cost_events = deque(maxlen=100000)  # Track last 100k events
        self.budgets = {}  # User/service budgets
        self.cost_alerts = []  # Active cost alerts
        
        # Current pricing (approximate as of 2025)
        self.pricing = {
            "gpt-4-turbo": {
                "input": 0.01 / 1000,  # $0.01 per 1K tokens
                "output": 0.03 / 1000  # $0.03 per 1K tokens
            },
            "gpt-3.5-turbo": {
                "input": 0.0005 / 1000,  # $0.0005 per 1K tokens
                "output": 0.0015 / 1000  # $0.0015 per 1K tokens
            },
            "text-embedding-3-small": 0.00002 / 1000,  # $0.00002 per 1K tokens
            "whisper": 0.006 / 60,  # $0.006 per minute
            "dalle-3": 0.04,  # $0.04 per image
            "gpt-4-vision": 0.01 / 1000  # $0.01 per 1K tokens
        }
        
        # Optimization cache
        self.response_cache = {}  # Query hash -> response
        self.embedding_cache = {}  # Text hash -> embedding
    
    def track_cost(self, service: str, operation: str, tokens_input: int = 0, tokens_output: int = 0, user_id: str = "", details: Dict[str, Any] = None):
        """Track cost for an operation."""
        cost = 0.0
        
        if service in self.pricing:
            pricing = self.pricing[service]
            if isinstance(pricing, dict):
                cost = (tokens_input * pricing.get("input", 0) + 
                       tokens_output * pricing.get("output", 0))
            else:
                cost = tokens_input * pricing
        
        event = CostEvent(
            timestamp=datetime.now(),
            service=service,
            operation=operation,
            cost=cost,
            tokens=tokens_input + tokens_output,
            user_id=user_id,
            details=details or {}
        )
        
        self.cost_events.append(event)
        
        # Check budgets
        self._check_budgets(user_id, cost)
        
        return cost
    
    def set_budget(self, identifier: str, amount: float, period: str = "monthly"):
        """Set budget for user or service."""
        self.budgets[identifier] = {
            "amount": amount,
            "period": period,
            "alerts_sent": []
        }
    
    def _check_budgets(self, user_id: str, new_cost: float):
        """Check if any budgets are exceeded."""
        # Check user budget
        if user_id and user_id in self.budgets:
            self._check_single_budget(user_id, new_cost)
        
        # Check global budget
        if "global" in self.budgets:
            self._check_single_budget("global", new_cost)
    
    def _check_single_budget(self, identifier: str, new_cost: float):
        """Check a single budget."""
        budget_info = self.budgets[identifier]
        period = budget_info["period"]
        
        # Calculate period start
        now = datetime.now()
        if period == "monthly":
            period_start = now.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
        elif period == "daily":
            period_start = now.replace(hour=0, minute=0, second=0, microsecond=0)
        elif period == "hourly":
            period_start = now.replace(minute=0, second=0, microsecond=0)
        else:
            return  # Unknown period
        
        # Calculate current spend
        current_spend = sum(
            event.cost for event in self.cost_events
            if (event.user_id == identifier or identifier == "global") and
               event.timestamp >= period_start
        )
        
        budget_amount = budget_info["amount"]
        utilization = current_spend / budget_amount
        
        # Send alerts at 50%, 80%, and 100%
        alerts_sent = budget_info["alerts_sent"]
        
        if utilization >= 1.0 and "100%" not in alerts_sent:
            self._send_budget_alert(identifier, "100%", current_spend, budget_amount)
            alerts_sent.append("100%")
        elif utilization >= 0.8 and "80%" not in alerts_sent:
            self._send_budget_alert(identifier, "80%", current_spend, budget_amount)
            alerts_sent.append("80%")
        elif utilization >= 0.5 and "50%" not in alerts_sent:
            self._send_budget_alert(identifier, "50%", current_spend, budget_amount)
            alerts_sent.append("50%")
    
    def _send_budget_alert(self, identifier: str, threshold: str, current_spend: float, budget: float):
        """Send budget alert."""
        alert = {
            "timestamp": datetime.now(),
            "identifier": identifier,
            "threshold": threshold,
            "current_spend": current_spend,
            "budget": budget,
            "utilization": current_spend / budget
        }
        
        self.cost_alerts.append(alert)
        print(f"🚨 Budget Alert: {identifier} has used {threshold} of budget (${current_spend:.4f} / ${budget:.2f})")
    
    def get_cost_summary(self, user_id: str = None, hours: int = 24) -> Dict[str, Any]:
        """Get cost summary for a period."""
        cutoff = datetime.now() - timedelta(hours=hours)
        
        events = [
            event for event in self.cost_events
            if event.timestamp >= cutoff and
               (not user_id or event.user_id == user_id)
        ]
        
        if not events:
            return {"total_cost": 0, "breakdown": {}, "token_usage": {}}
        
        # Cost breakdown by service
        service_costs = defaultdict(float)
        service_tokens = defaultdict(int)
        
        for event in events:
            service_costs[event.service] += event.cost
            service_tokens[event.service] += event.tokens
        
        total_cost = sum(service_costs.values())
        total_tokens = sum(service_tokens.values())
        
        return {
            "period_hours": hours,
            "user_id": user_id,
            "total_cost": total_cost,
            "total_tokens": total_tokens,
            "breakdown": dict(service_costs),
            "token_usage": dict(service_tokens),
            "cost_per_token": total_cost / total_tokens if total_tokens > 0 else 0,
            "event_count": len(events)
        }
    
    def optimize_query(self, query_hash: str, response: Any) -> bool:
        """Cache response for optimization."""
        if query_hash not in self.response_cache:
            self.response_cache[query_hash] = {
                "response": response,
                "timestamp": datetime.now(),
                "hit_count": 1
            }
            return False  # Not cached
        else:
            self.response_cache[query_hash]["hit_count"] += 1
            return True  # Cache hit
    
    def get_optimization_stats(self) -> Dict[str, Any]:
        """Get cache optimization statistics."""
        if not self.response_cache:
            return {"cache_size": 0, "hit_rate": 0, "total_hits": 0}
        
        total_hits = sum(item["hit_count"] for item in self.response_cache.values())
        cache_hits = sum(item["hit_count"] - 1 for item in self.response_cache.values())
        hit_rate = cache_hits / total_hits if total_hits > 0 else 0
        
        return {
            "cache_size": len(self.response_cache),
            "total_requests": total_hits,
            "cache_hits": cache_hits,
            "hit_rate": hit_rate,
            "cost_savings_estimate": cache_hits * 0.002  # Rough estimate
        }

# Initialize cost manager
cost_manager = CostManager()

# Set sample budgets
cost_manager.set_budget("global", 100.0, "monthly")  # $100/month global budget
cost_manager.set_budget(user_id, 10.0, "daily")  # $10/day per user

print("💰 Cost management system initialized")
print(f"   Global budget: $100/month")
print(f"   User budget: $10/day")

### 🌐 Production RAG Service

In [ ]:
import asyncio
from concurrent.futures import ThreadPoolExecutor
from typing import Callable, Awaitable

class HealthCheck:
    """Health check system for production RAG."""
    
    def __init__(self):
        self.checks = {}
        self.status = "healthy"
        self.last_check = None
    
    def register_check(self, name: str, check_func: Callable[[], bool], critical: bool = True):
        """Register a health check."""
        self.checks[name] = {
            "func": check_func,
            "critical": critical,
            "last_result": None,
            "last_check": None
        }
    
    def run_checks(self) -> Dict[str, Any]:
        """Run all health checks."""
        results = {}
        all_healthy = True
        
        for name, check_info in self.checks.items():
            try:
                start_time = time.time()
                result = check_info["func"]()
                duration = time.time() - start_time
                
                results[name] = {
                    "healthy": result,
                    "duration": duration,
                    "critical": check_info["critical"]
                }
                
                check_info["last_result"] = result
                check_info["last_check"] = datetime.now()
                
                if not result and check_info["critical"]:
                    all_healthy = False
                    
            except Exception as e:
                results[name] = {
                    "healthy": False,
                    "error": str(e),
                    "critical": check_info["critical"]
                }
                
                if check_info["critical"]:
                    all_healthy = False
        
        self.status = "healthy" if all_healthy else "unhealthy"
        self.last_check = datetime.now()
        
        return {
            "status": self.status,
            "timestamp": self.last_check.isoformat(),
            "checks": results
        }

class ProductionRAGService:
    """Production-ready RAG service with all enterprise features."""
    
    def __init__(self):
        # Core components
        self.metrics = RAGMetrics()
        self.security = SecurityManager()
        self.cost_manager = CostManager()
        self.health_check = HealthCheck()
        
        # Service configuration
        self.config = {
            "max_concurrent_requests": 100,
            "request_timeout": 30,
            "cache_enabled": True,
            "debug_mode": False
        }
        
        # Request handling
        self.executor = ThreadPoolExecutor(max_workers=10)
        self.active_requests = {}
        
        # Initialize health checks
        self._setup_health_checks()
        
        # Startup tasks
        self._initialize_service()
    
    def _setup_health_checks(self):
        """Setup health checks for the service."""
        self.health_check.register_check(
            "service_ready",
            lambda: True,  # Always ready for demo
            critical=True
        )
        
        self.health_check.register_check(
            "database_connection",
            lambda: True,  # Mock database check
            critical=True
        )
        
        self.health_check.register_check(
            "api_keys_valid",
            lambda: bool(os.getenv("OPENAI_API_KEY")),
            critical=True
        )
        
        self.health_check.register_check(
            "memory_usage",
            lambda: len(self.active_requests) < self.config["max_concurrent_requests"],
            critical=False
        )
    
    def _initialize_service(self):
        """Initialize the service."""
        print("🚀 Initializing Production RAG Service...")
        
        # Run initial health check
        health_status = self.health_check.run_checks()
        if health_status["status"] != "healthy":
            raise RuntimeError(f"Service unhealthy: {health_status}")
        
        print("   ✅ Health checks passed")
        print("   🔐 Security system ready")
        print("   💰 Cost tracking enabled")
        print("   📊 Metrics collection active")
        print("🎉 Production RAG Service ready!")
    
    async def query(self, query: str, user_token: str, **kwargs) -> Dict[str, Any]:
        """Main query endpoint with full production features."""
        request_id = str(uuid.uuid4())
        start_time = time.time()
        
        try:
            # Authentication
            user = self._authenticate_request(user_token)
            if not user:
                return {"error": "Authentication failed", "code": 401}
            
            # Authorization
            if not self.security.has_permission(user, "query"):
                return {"error": "Insufficient permissions", "code": 403}
            
            # Rate limiting
            if not self.security.check_rate_limit(user.id):
                return {"error": "Rate limit exceeded", "code": 429}
            
            # Track active request
            self.active_requests[request_id] = {
                "user_id": user.id,
                "query": query,
                "start_time": start_time
            }
            
            # Process query
            with self.metrics.timer("query.total_time"):
                result = await self._process_query(query, user, request_id, **kwargs)
            
            # Track success
            duration = time.time() - start_time
            self.metrics.record_query_performance(
                duration=duration,
                success=True,
                tokens_used=result.get("tokens_used", 0)
            )
            
            # Track costs
            self.cost_manager.track_cost(
                service="rag_query",
                operation="query",
                tokens_input=result.get("input_tokens", 0),
                tokens_output=result.get("output_tokens", 0),
                user_id=user.id
            )
            
            return result
            
        except Exception as e:
            duration = time.time() - start_time
            self.metrics.record_query_performance(duration, False)
            
            # Log error
            self.security._log_audit(
                user_id=user.id if 'user' in locals() else "unknown",
                action="query_error",
                resource="query",
                details={"error": str(e), "query": query}
            )
            
            return {"error": "Internal server error", "code": 500}
            
        finally:
            # Clean up
            if request_id in self.active_requests:
                del self.active_requests[request_id]
    
    def _authenticate_request(self, token: str) -> Optional[User]:
        """Authenticate request token."""
        # Try session token first
        user = self.security.validate_session(token)
        if user:
            return user
        
        # Try API key
        user = self.security.validate_api_key(token)
        return user
    
    async def _process_query(self, query: str, user: User, request_id: str, **kwargs) -> Dict[str, Any]:
        """Process the actual query (mock implementation)."""
        # Simulate processing time
        await asyncio.sleep(0.1)
        
        # Mock response
        response = {
            "answer": f"This is a mock response to the query: '{query}'. In a real implementation, this would be processed by your RAG system.",
            "sources": [
                {"title": "Mock Source 1", "url": "https://example.com/doc1"},
                {"title": "Mock Source 2", "url": "https://example.com/doc2"}
            ],
            "confidence": 0.85,
            "request_id": request_id,
            "processing_time": 0.1,
            "tokens_used": 150,
            "input_tokens": 50,
            "output_tokens": 100
        }
        
        return response
    
    def get_status(self) -> Dict[str, Any]:
        """Get service status and metrics."""
        health_status = self.health_check.run_checks()
        metrics_summary = self.metrics.get_summary()
        cost_summary = self.cost_manager.get_cost_summary()
        optimization_stats = self.cost_manager.get_optimization_stats()
        
        return {
            "service": "Production RAG",
            "version": "2025.1",
            "health": health_status,
            "metrics": metrics_summary,
            "costs": cost_summary,
            "optimization": optimization_stats,
            "active_requests": len(self.active_requests),
            "config": self.config
        }
    
    def get_user_analytics(self, user_id: str, hours: int = 24) -> Dict[str, Any]:
        """Get analytics for a specific user."""
        cost_summary = self.cost_manager.get_cost_summary(user_id, hours)
        audit_logs = self.security.get_audit_logs(user_id, hours)
        
        return {
            "user_id": user_id,
            "period_hours": hours,
            "costs": cost_summary,
            "activity_count": len(audit_logs),
            "last_activities": [
                {
                    "action": log.action,
                    "timestamp": log.timestamp.isoformat(),
                    "resource": log.resource
                }
                for log in audit_logs[-10:]  # Last 10 activities
            ]
        }
    
    async def batch_query(self, queries: List[str], user_token: str) -> List[Dict[str, Any]]:
        """Process multiple queries efficiently."""
        tasks = [self.query(query, user_token) for query in queries]
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        # Convert exceptions to error responses
        processed_results = []
        for result in results:
            if isinstance(result, Exception):
                processed_results.append({"error": str(result), "code": 500})
            else:
                processed_results.append(result)
        
        return processed_results
    
    def shutdown(self):
        """Gracefully shutdown the service."""
        print("🛑 Shutting down Production RAG Service...")
        
        # Wait for active requests to complete
        while self.active_requests:
            print(f"   Waiting for {len(self.active_requests)} active requests...")
            time.sleep(1)
        
        # Shutdown executor
        self.executor.shutdown(wait=True)
        
        print("✅ Service shutdown complete")

# Initialize production service
prod_rag_service = ProductionRAGService()

print("\n🏭 Production RAG Service initialized successfully!")

## 🧪 Production Service Demo

In [ ]:
# Demo the production service
print("🎮 Production RAG Service Demo")
print("=" * 50)

# Create a session for our admin user
admin_session = prod_rag_service.security.create_session(admin_id)
print(f"👤 Created admin session: {admin_session[:8]}...")

# Test single query
print("\n🔍 Testing single query...")
test_query = "What are the key components of a production RAG system?"

# Note: In Jupyter, we need to handle async differently
from asyncio import create_task, get_event_loop

try:
    loop = get_event_loop()
except RuntimeError:
    import asyncio
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

# Run the query
result = loop.run_until_complete(
    prod_rag_service.query(test_query, admin_session)
)

if "error" not in result:
    print(f"✅ Query successful!")
    print(f"   Request ID: {result['request_id']}")
    print(f"   Answer: {result['answer'][:100]}...")
    print(f"   Tokens used: {result['tokens_used']}")
    print(f"   Processing time: {result['processing_time']}s")
else:
    print(f"❌ Query failed: {result['error']}")

# Test batch queries
print("\n📦 Testing batch queries...")
batch_queries = [
    "What is retrieval-augmented generation?",
    "How do you scale RAG systems?",
    "What are the security considerations for RAG?"
]

batch_results = loop.run_until_complete(
    prod_rag_service.batch_query(batch_queries, admin_session)
)

print(f"✅ Batch processing complete: {len(batch_results)} queries")
for i, result in enumerate(batch_results):
    if "error" not in result:
        print(f"   Query {i+1}: Success ({result['tokens_used']} tokens)")
    else:
        print(f"   Query {i+1}: Failed - {result['error']}")

In [ ]:
# Check service status
print("\n📊 Service Status Report")
print("=" * 30)

status = prod_rag_service.get_status()

print(f"Service: {status['service']} v{status['version']}")
print(f"Health: {status['health']['status']}")
print(f"Active Requests: {status['active_requests']}")

print("\n📈 Performance Metrics:")
perf = status['metrics']['performance']
print(f"   Total Queries: {perf['total_queries']}")
print(f"   Success Rate: {perf['success_rate']:.1%}")
print(f"   Avg Response Time: {perf['avg_query_time']:.3f}s")

print("\n💰 Cost Summary:")
costs = status['costs']
print(f"   Total Cost (24h): ${costs['total_cost']:.4f}")
print(f"   Total Tokens: {costs['total_tokens']:,}")
print(f"   Cost per Token: ${costs['cost_per_token']:.6f}")

print("\n⚡ Optimization:")
opt = status['optimization']
print(f"   Cache Size: {opt['cache_size']}")
print(f"   Hit Rate: {opt['hit_rate']:.1%}")
print(f"   Estimated Savings: ${opt['cost_savings_estimate']:.4f}")

print("\n🔍 Health Checks:")
for check_name, check_result in status['health']['checks'].items():
    status_icon = "✅" if check_result['healthy'] else "❌"
    critical_text = "(Critical)" if check_result['critical'] else ""
    print(f"   {status_icon} {check_name} {critical_text}")
    if 'duration' in check_result:
        print(f"      Duration: {check_result['duration']:.3f}s")

In [ ]:
# Get user analytics
print("\n👤 User Analytics")
print("=" * 20)

user_analytics = prod_rag_service.get_user_analytics(admin_id)

print(f"User ID: {user_analytics['user_id']}")
print(f"Period: {user_analytics['period_hours']} hours")

print("\n💰 User Costs:")
user_costs = user_analytics['costs']
print(f"   Total Cost: ${user_costs['total_cost']:.4f}")
print(f"   Total Tokens: {user_costs['total_tokens']:,}")
print(f"   Queries: {user_costs['event_count']}")

if user_costs['breakdown']:
    print("   Breakdown:")
    for service, cost in user_costs['breakdown'].items():
        print(f"     {service}: ${cost:.4f}")

print(f"\n📝 Recent Activity ({user_analytics['activity_count']} total):")
for activity in user_analytics['last_activities']:
    timestamp = datetime.fromisoformat(activity['timestamp']).strftime('%H:%M:%S')
    print(f"   {timestamp} - {activity['action']} on {activity['resource']}")

## 🚀 Deployment Strategies

In [ ]:
# Docker deployment configuration
dockerfile_content = '''
# Production RAG Service Dockerfile
FROM python:3.11-slim

WORKDIR /app

# Install system dependencies
RUN apt-get update && apt-get install -y \
    gcc \
    g++ \
    && rm -rf /var/lib/apt/lists/*

# Install Python dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy application code
COPY . .

# Create non-root user
RUN useradd -m -u 1001 raguser
USER raguser

# Health check
HEALTHCHECK --interval=30s --timeout=10s --start-period=60s --retries=3 \
    CMD python health_check.py

# Expose port
EXPOSE 8000

# Run the application
CMD ["python", "main.py"]
'''

# Kubernetes deployment configuration
k8s_deployment = '''
apiVersion: apps/v1
kind: Deployment
metadata:
  name: production-rag
  labels:
    app: production-rag
spec:
  replicas: 3
  selector:
    matchLabels:
      app: production-rag
  template:
    metadata:
      labels:
        app: production-rag
    spec:
      containers:
      - name: rag-service
        image: your-registry/production-rag:latest
        ports:
        - containerPort: 8000
        env:
        - name: OPENAI_API_KEY
          valueFrom:
            secretKeyRef:
              name: rag-secrets
              key: openai-api-key
        - name: DATABASE_URL
          valueFrom:
            secretKeyRef:
              name: rag-secrets
              key: database-url
        resources:
          requests:
            memory: "512Mi"
            cpu: "250m"
          limits:
            memory: "1Gi"
            cpu: "500m"
        livenessProbe:
          httpGet:
            path: /health
            port: 8000
          initialDelaySeconds: 60
          periodSeconds: 30
        readinessProbe:
          httpGet:
            path: /ready
            port: 8000
          initialDelaySeconds: 30
          periodSeconds: 10
---
apiVersion: v1
kind: Service
metadata:
  name: production-rag-service
spec:
  selector:
    app: production-rag
  ports:
  - protocol: TCP
    port: 80
    targetPort: 8000
  type: LoadBalancer
---
apiVersion: autoscaling/v2
kind: HorizontalPodAutoscaler
metadata:
  name: production-rag-hpa
spec:
  scaleTargetRef:
    apiVersion: apps/v1
    kind: Deployment
    name: production-rag
  minReplicas: 3
  maxReplicas: 10
  metrics:
  - type: Resource
    resource:
      name: cpu
      target:
        type: Utilization
        averageUtilization: 70
  - type: Resource
    resource:
      name: memory
      target:
        type: Utilization
        averageUtilization: 80
'''

# CI/CD pipeline (GitHub Actions)
cicd_pipeline = '''
name: Deploy Production RAG

on:
  push:
    branches: [main]
  pull_request:
    branches: [main]

jobs:
  test:
    runs-on: ubuntu-latest
    steps:
    - uses: actions/checkout@v3
    - name: Set up Python
      uses: actions/setup-python@v4
      with:
        python-version: '3.11'
    - name: Install dependencies
      run: |
        pip install -r requirements.txt
        pip install pytest pytest-cov
    - name: Run tests
      run: |
        pytest tests/ --cov=src --cov-report=xml
    - name: Upload coverage
      uses: codecov/codecov-action@v3
  
  build-and-deploy:
    needs: test
    runs-on: ubuntu-latest
    if: github.ref == 'refs/heads/main'
    steps:
    - uses: actions/checkout@v3
    - name: Configure AWS credentials
      uses: aws-actions/configure-aws-credentials@v2
      with:
        aws-access-key-id: ${{ secrets.AWS_ACCESS_KEY_ID }}
        aws-secret-access-key: ${{ secrets.AWS_SECRET_ACCESS_KEY }}
        aws-region: us-west-2
    
    - name: Login to Amazon ECR
      uses: aws-actions/amazon-ecr-login@v1
    
    - name: Build and push Docker image
      run: |
        docker build -t production-rag .
        docker tag production-rag:latest $ECR_REGISTRY/production-rag:latest
        docker push $ECR_REGISTRY/production-rag:latest
    
    - name: Deploy to EKS
      run: |
        aws eks update-kubeconfig --name production-cluster
        kubectl set image deployment/production-rag rag-service=$ECR_REGISTRY/production-rag:latest
        kubectl rollout status deployment/production-rag
'''

print("🚀 Production Deployment Configurations Generated")
print("\n📁 Files created:")
print("   📄 Dockerfile - Container configuration")
print("   ☸️  k8s-deployment.yaml - Kubernetes deployment")
print("   🔄 .github/workflows/deploy.yml - CI/CD pipeline")

print("\n🏗️ Deployment Strategy Recommendations:")
print("   1. **Development**: Docker Compose for local development")
print("   2. **Staging**: Single Kubernetes cluster with resource limits")
print("   3. **Production**: Multi-region Kubernetes with auto-scaling")
print("   4. **Enterprise**: Service mesh (Istio) + GitOps (ArgoCD)")

print("\n📊 Monitoring Stack:")
print("   • Prometheus + Grafana for metrics")
print("   • ELK Stack for logging")
print("   • Jaeger for distributed tracing")
print("   • PagerDuty for alerting")

print("\n🔒 Security Considerations:")
print("   • Network policies for pod-to-pod communication")
print("   • RBAC for Kubernetes access control")
print("   • Vault for secrets management")
print("   • Regular security scanning of images")

## 📊 Advanced Monitoring Dashboard

In [ ]:
class MonitoringDashboard:
    """Advanced monitoring dashboard for production RAG."""
    
    def __init__(self, rag_service: ProductionRAGService):
        self.service = rag_service
        self.alert_rules = []
        self.dashboards = {}
    
    def generate_dashboard(self) -> str:
        """Generate monitoring dashboard configuration."""
        
        grafana_dashboard = {
            "dashboard": {
                "title": "Production RAG Monitoring",
                "tags": ["rag", "production", "ai"],
                "panels": [
                    {
                        "title": "Request Rate",
                        "type": "graph",
                        "targets": [{
                            "expr": "rate(rag_requests_total[5m])",
                            "legendFormat": "Requests/sec"
                        }]
                    },
                    {
                        "title": "Response Time",
                        "type": "graph",
                        "targets": [{
                            "expr": "histogram_quantile(0.95, rag_request_duration_seconds_bucket)",
                            "legendFormat": "95th percentile"
                        }]
                    },
                    {
                        "title": "Error Rate",
                        "type": "singlestat",
                        "targets": [{
                            "expr": "rate(rag_requests_total{status=~'5..'}[5m]) / rate(rag_requests_total[5m])",
                            "legendFormat": "Error Rate"
                        }]
                    },
                    {
                        "title": "Cost per Hour",
                        "type": "graph",
                        "targets": [{
                            "expr": "rate(rag_cost_total[1h])",
                            "legendFormat": "$/hour"
                        }]
                    },
                    {
                        "title": "Token Usage",
                        "type": "graph",
                        "targets": [{
                            "expr": "rate(rag_tokens_total[5m])",
                            "legendFormat": "Tokens/sec"
                        }]
                    },
                    {
                        "title": "Cache Hit Rate",
                        "type": "singlestat",
                        "targets": [{
                            "expr": "rag_cache_hits_total / rag_cache_requests_total",
                            "legendFormat": "Hit Rate"
                        }]
                    }
                ]
            }
        }
        
        return json.dumps(grafana_dashboard, indent=2)
    
    def create_alert_rules(self) -> List[Dict[str, Any]]:
        """Create alert rules for the RAG system."""
        
        alert_rules = [
            {
                "alert": "RAGHighErrorRate",
                "expr": "rate(rag_requests_total{status=~'5..'}[5m]) / rate(rag_requests_total[5m]) > 0.05",
                "for": "5m",
                "labels": {"severity": "critical"},
                "annotations": {
                    "summary": "RAG service has high error rate",
                    "description": "Error rate is {{ $value | humanizePercentage }} over the last 5 minutes"
                }
            },
            {
                "alert": "RAGSlowResponse",
                "expr": "histogram_quantile(0.95, rag_request_duration_seconds_bucket) > 5",
                "for": "10m",
                "labels": {"severity": "warning"},
                "annotations": {
                    "summary": "RAG service response time is slow",
                    "description": "95th percentile response time is {{ $value }}s"
                }
            },
            {
                "alert": "RAGHighCost",
                "expr": "rate(rag_cost_total[1h]) > 10",
                "for": "15m",
                "labels": {"severity": "warning"},
                "annotations": {
                    "summary": "RAG service cost is high",
                    "description": "Cost is ${{ $value }}/hour"
                }
            },
            {
                "alert": "RAGServiceDown",
                "expr": "up{job='rag-service'} == 0",
                "for": "1m",
                "labels": {"severity": "critical"},
                "annotations": {
                    "summary": "RAG service is down",
                    "description": "RAG service has been down for more than 1 minute"
                }
            }
        ]
        
        return alert_rules
    
    def generate_slo_config(self) -> Dict[str, Any]:
        """Generate Service Level Objectives configuration."""
        
        slo_config = {
            "slos": [
                {
                    "name": "rag-availability",
                    "description": "RAG service availability",
                    "service": "rag-service",
                    "slo": 99.9,  # 99.9% availability
                    "sli": {
                        "events": {
                            "error_query": "sum(rate(rag_requests_total{status=~'5..'}[5m]))",
                            "total_query": "sum(rate(rag_requests_total[5m]))"
                        }
                    },
                    "alerting": {
                        "burn_rates": {
                            "critical": {"1h": 14.4, "5m": 14.4},
                            "warning": {"6h": 6, "30m": 6}
                        }
                    }
                },
                {
                    "name": "rag-latency",
                    "description": "RAG service latency",
                    "service": "rag-service",
                    "slo": 95,  # 95% of requests under 2s
                    "sli": {
                        "events": {
                            "good_query": "sum(rate(rag_request_duration_seconds_bucket{le='2.0'}[5m]))",
                            "total_query": "sum(rate(rag_request_duration_seconds_count[5m]))"
                        }
                    }
                }
            ]
        }
        
        return slo_config

# Create monitoring dashboard
dashboard = MonitoringDashboard(prod_rag_service)

print("📊 Advanced Monitoring Configuration")
print("=" * 40)

# Generate configurations
grafana_config = dashboard.generate_dashboard()
alert_rules = dashboard.create_alert_rules()
slo_config = dashboard.generate_slo_config()

print("✅ Generated monitoring configurations:")
print(f"   📊 Grafana Dashboard: {len(json.loads(grafana_config)['dashboard']['panels'])} panels")
print(f"   🚨 Alert Rules: {len(alert_rules)} rules")
print(f"   🎯 SLO Configuration: {len(slo_config['slos'])} objectives")

print("\n🎯 Service Level Objectives:")
for slo in slo_config['slos']:
    print(f"   • {slo['name']}: {slo['slo']}% {slo['description']}")

print("\n🚨 Alert Rules:")
for rule in alert_rules:
    severity_icon = "🔥" if rule['labels']['severity'] == 'critical' else "⚠️"
    print(f"   {severity_icon} {rule['alert']}: {rule['annotations']['summary']}")

print("\n📈 Key Metrics Tracked:")
print("   • Request rate (RPS)")
print("   • Response time percentiles")
print("   • Error rates by status code")
print("   • Cost per hour/day/month")
print("   • Token usage and efficiency")
print("   • Cache hit rates")
print("   • System resource usage")
print("   • User activity patterns")

print("\n🔧 Monitoring Best Practices:")
print("   1. Set up alerting for SLO violations")
print("   2. Monitor both technical and business metrics")
print("   3. Use distributed tracing for complex queries")
print("   4. Track cost metrics to prevent surprises")
print("   5. Monitor cache efficiency for optimization")
print("   6. Set up automated runbooks for common issues")

## 🎓 Key Takeaways: Production RAG

### ✅ What You've Built

1. **Comprehensive Metrics System** - Performance, cost, and business metrics
2. **Security & Access Control** - Authentication, authorization, audit trails
3. **Cost Management** - Budget tracking, optimization, alerts
4. **Health Monitoring** - Service health checks and status reporting
5. **Production Service** - Enterprise-grade RAG service with all features
6. **Deployment Configurations** - Docker, Kubernetes, CI/CD pipelines
7. **Advanced Monitoring** - Dashboards, alerts, SLOs

### 🏗️ Production Architecture Principles

**Scalability:**
- Horizontal scaling with load balancers
- Auto-scaling based on demand
- Caching for performance optimization
- Async processing for non-blocking operations

**Reliability:**
- Health checks and circuit breakers
- Graceful degradation during failures
- Retry logic with exponential backoff
- Multi-region deployments for disaster recovery

**Security:**
- Multiple authentication methods (sessions, API keys)
- Role-based access control (RBAC)
- Rate limiting and DDoS protection
- Comprehensive audit logging

**Observability:**
- Structured logging with correlation IDs
- Metrics collection and alerting
- Distributed tracing for complex requests
- Service Level Objectives (SLOs) monitoring

### 💰 Cost Optimization Strategies

**1. Intelligent Caching**
- Cache frequently asked questions
- Cache embeddings for repeated content
- Implement cache warming for predictable queries

**2. Model Selection**
- Use smaller models for simple queries
- Route complex queries to more capable models
- Implement model fallback strategies

**3. Token Optimization**
- Compress context windows intelligently
- Use retrieval to focus on relevant content only
- Implement token counting and limits

**4. Usage Patterns**
- Batch processing for bulk operations
- Off-peak processing for non-urgent tasks
- User-based budgets and quotas

### 🚀 Deployment Best Practices

**Container Strategy:**
- Multi-stage Docker builds for smaller images
- Security scanning in CI/CD pipeline
- Non-root user containers
- Resource limits and health checks

**Kubernetes Deployment:**
- HPA for automatic scaling
- Pod disruption budgets for availability
- Network policies for security
- Secrets management with external systems

**CI/CD Pipeline:**
- Automated testing at multiple levels
- Security and vulnerability scanning
- Blue-green or canary deployments
- Rollback capabilities

### 📊 Monitoring & Alerting

**Key Metrics:**
- **RED Metrics**: Rate, Errors, Duration
- **USE Metrics**: Utilization, Saturation, Errors
- **Business Metrics**: Cost, User satisfaction, Feature usage

**Alert Strategy:**
- SLO-based alerting to reduce noise
- Severity levels with appropriate escalation
- Runbooks for common incident response
- Post-mortem processes for learning

### 🔮 Future Considerations

**Emerging Technologies:**
- Edge deployment for latency reduction
- Federated learning for privacy-preserving improvements
- Quantum-resistant encryption preparation
- Advanced AI for self-healing systems

**Scalability Evolution:**
- Microservices architecture for complex domains
- Event-driven architectures for real-time processing
- Multi-cloud strategies for resilience
- Serverless components for cost efficiency

Congratulations! You now have comprehensive knowledge of building, deploying, and operating production-grade RAG systems! 🎉

### 📚 Your Complete RAG Journey

1. ✅ **RAG Concepts** - Understood fundamentals and evolution
2. ✅ **Modern Implementation** - Built advanced RAG with LangGraph
3. ✅ **Conversational Systems** - Added memory and context
4. ✅ **Multimodal Capabilities** - Handled images, audio, video
5. ✅ **Production Deployment** - Enterprise-grade systems

You're now ready to build and deploy sophisticated RAG applications that can scale to serve millions of users! 🚀